In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kianindeed/imdb-movie-dataset-dec-2023")

print("Path to dataset files:", path)

100%|██████████| 106k/106k [00:00<00:00, 19.1MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/kianindeed/imdb-movie-dataset-dec-2023/versions/1


In [ ]:
import kagglehub
import pandas as pd

# Download latest version
path = kagglehub.dataset_download("kianindeed/imdb-movie-dataset-dec-2023")

print("Path to dataset files:", path)

# Load the dataset
# Use the path returned by kagglehub.dataset_download to construct the correct file path
file_path = path + "/imdb_movie_data_2023.csv"  # Assuming the CSV file is named 'imdb_movie_data_2023.csv' within the downloaded dataset folder
df = pd.read_csv(file_path)

# Display basic info
print(df.info())

# Show first few rows
df.head()


Path to dataset files: /root/.cache/kagglehub/datasets/kianindeed/imdb-movie-dataset-dec-2023/versions/1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1950 entries, 0 to 1949
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  1950 non-null   int64  
 1   Moive Name  1950 non-null   object 
 2   Rating      1944 non-null   float64
 3   Votes       1944 non-null   float64
 4   Meta Score  1833 non-null   float64
 5   Genre       1833 non-null   object 
 6   PG Rating   1874 non-null   object 
 7   Year        1950 non-null   int64  
 8   Duration    1948 non-null   object 
 9   Cast        1833 non-null   object 
 10  Director    1833 non-null   object 
dtypes: float64(3), int64(2), object(6)
memory usage: 167.7+ KB
None


,Unnamed: 0,Moive Name,Rating,Votes,Meta Score,Genre,PG Rating,Year,Duration,Cast,Director
0,0,Leave the World Behind,6.5,90000.0,67.0,"Drama, Mystery, Thriller",R,2023,2h 18m,"Julia Roberts, Mahershala Ali, Ethan Hawke, My...",Sam Esmail
1,1,Wonka,7.4,24000.0,66.0,"Adventure, Comedy, Family",PG,2023,1h 56m,"Timothée Chalamet, Gustave Die, Murray McArthu...",Paul King
2,2,Poor Things,8.5,6700.0,86.0,"Comedy, Drama, Romance",R,2023,2h 21m,"Emma Stone, Mark Ruffalo, Willem Dafoe, Ramy Y...",Yorgos Lanthimos
3,3,Killers of the Flower Moon,7.8,128000.0,89.0,"Crime, Drama, History",R,2023,3h 26m,"Leonardo DiCaprio, Robert De Niro, Lily Gladst...",Martin Scorsese
4,4,May December,7.0,21000.0,85.0,"Comedy, Drama",R,2023,1h 57m,"Natalie Portman, Chris Tenzis, Charles Melton,...",Todd Haynes


In [ ]:
# Display column names
print(df.columns)


Index(['Unnamed: 0', 'Moive Name', 'Rating', 'Votes', 'Meta Score', 'Genre',
       'PG Rating', 'Year', 'Duration', 'Cast', 'Director'],
      dtype='object')


In [ ]:
# Drop unnecessary columns, if they exist
if 'Unnamed: 0' in df.columns:
    df = df.drop(columns=["Unnamed: 0"])
else:
    print("Column 'Unnamed: 0' not found in DataFrame.")

# Continue with the rest of your code
print(df.isnull().sum())

# Fill or drop missing values (if needed)
df = df.dropna()  # OR fill with appropriate values if needed

Column 'Unnamed: 0' not found in DataFrame.
Moive Name    0
Rating        0
Votes         0
Meta Score    0
Genre         0
PG Rating     0
Year          0
Duration      0
Cast          0
Director      0
dtype: int64


In [ ]:
# Check percentage of missing values
missing_percentage = df.isnull().sum() / len(df) * 100
print(missing_percentage)


Moive Name    0.0
Rating        0.0
Votes         0.0
Meta Score    0.0
Genre         0.0
PG Rating     0.0
Year          0.0
Duration      0.0
Cast          0.0
Director      0.0
dtype: float64


In [ ]:
df["Meta Score"].fillna(df["Meta Score"].median(), inplace=True)  # Filling with median score
df["PG Rating"].fillna("Not Rated", inplace=True)  # Filling categorical values


<ipython-input-7-c5a9deff7ee9>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Meta Score"].fillna(df["Meta Score"].median(), inplace=True)  # Filling with median score
<ipython-input-7-c5a9deff7ee9>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[

In [ ]:
# Split genres into separate categories
df["Genre"] = df["Genre"].str.split(", ")  # Convert genre string into a list

# Convert genres into individual columns (One-Hot Encoding)
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
genres_encoded = pd.DataFrame(mlb.fit_transform(df["Genre"]), columns=mlb.classes_)

# Merge back into the main dataframe
df = df.join(genres_encoded)
df.drop(columns=["Genre"], inplace=True)  # Drop the original Genre column


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Convert Cast and Director into text format
df["Cast"] = df["Cast"].fillna("")  # Fill missing values
df["Director"] = df["Director"].fillna("")

# Combine Cast and Director into a single feature
df["Cast_Director"] = df["Cast"] + " " + df["Director"]

# Apply TF-IDF Vectorization
tfidf = TfidfVectorizer(stop_words="english")
tfidf_matrix = tfidf.fit_transform(df["Cast_Director"])


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Compute cosine similarity between TF-IDF vectors (Cast + Director)
cosine_sim_cast_director = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Compute cosine similarity for genres
cosine_sim_genre = cosine_similarity(genres_encoded, genres_encoded)

# Combine both similarity scores (weighted average)
cosine_sim = (cosine_sim_cast_director * 0.7) + (cosine_sim_genre * 0.3)


In [ ]:
def recommend_movies(movie_title, df, cosine_sim, top_n=10):
    # Get the index of the movie
    indices = pd.Series(df.index, index=df["Moive Name"]).drop_duplicates()

    if movie_title not in indices:
        return "Movie not found in dataset."

    idx = indices[movie_title]

    # Get similarity scores for all movies
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort movies based on similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the top N most similar movies (excluding the input movie)
    sim_scores = sim_scores[1:top_n+1]

    # Get movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top recommended movie names
    return df["Moive Name"].iloc[movie_indices].tolist()


In [ ]:
print(df["Moive Name"].head(10))  # Check first 10 movie titles


0                               Leave the World Behind
1                                                Wonka
2                                          Poor Things
3                           Killers of the Flower Moon
4                                         May December
5     The Hunger Games: The Ballad of Songbirds & S...
6                                             Napoleon
7                                          Oppenheimer
8                                        Love Actually
9                                      Candy Cane Lane
Name: Moive Name, dtype: object


In [ ]:
# Rename column if needed
df.rename(columns={"Moive Name": "Movie Name"}, inplace=True)

# Convert all movie names to lowercase and remove extra spaces
df["Movie Name"] = df["Movie Name"].str.strip().str.lower()


In [ ]:
print(df.columns)


Index(['Movie Name', 'Rating', 'Votes', 'Meta Score', 'PG Rating', 'Year',
       'Duration', 'Cast', 'Director', 'Action', 'Adventure', 'Animation',
       'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family',
       'Fantasy', 'History', 'Horror', 'Music', 'Musical', 'Mystery',
       'Romance', 'Sci-Fi', 'Sport', 'Thriller', 'War', 'Western',
       'Cast_Director'],
      dtype='object')


In [ ]:
def recommend_movies(movie_title, df, cosine_sim, top_n=10):
    # Convert input movie title to lowercase and strip spaces
    movie_title = movie_title.strip().lower()

    # Create a mapping of lowercase movie names to their indices
    indices = pd.Series(df.index, index=df["Movie Name"].str.lower()).drop_duplicates()

    # Check if movie exists
    if movie_title not in indices:
        return "Movie not found in dataset."

    idx = indices[movie_title]

    # Get similarity scores for all movies
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort movies based on similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the top N most similar movies (excluding the input movie)
    sim_scores = sim_scores[1:top_n+1]

    # Get movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top recommended movie names
    return df["Movie Name"].iloc[movie_indices].tolist()


In [ ]:
recommend_movies("inception", df, cosine_sim, top_n=5)


['hereditary',
 'beau is afraid',
 'brightburn',
 'the invisible man',
 "don't worry darling"]

In [ ]:
recommend_movies("top gun",df,cosine_sim, top_n=5)

['jojo rabbit', 'slap shot', 'champions', 'somewhere in queens', 'creed iii']

In [ ]:
import numpy as np

def recommend_movies(movie_title, df, cosine_sim, top_n=10):
    # Convert input movie title to lowercase and strip spaces
    movie_title = movie_title.strip().lower()

    # Create a mapping of lowercase movie names to their indices
    indices = pd.Series(df.index, index=df["Movie Name"].str.lower()).drop_duplicates()

    if movie_title not in indices:
        return "Movie not found in dataset."

    idx = indices[movie_title]

    # Get similarity scores for all movies
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort movies based on similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Filter based on high ratings and votes
    df_filtered = df.loc[movie_indices].copy()

    # Normalize ratings and votes
    df_filtered["Rating"] = df_filtered["Rating"].astype(float)
    df_filtered["Votes"] = df_filtered["Votes"].astype(int)

    max_rating = df_filtered["Rating"].max()
    max_votes = df_filtered["Votes"].max()

    df_filtered["Weighted_Score"] = (df_filtered["Rating"] / max_rating) * 0.7 + (df_filtered["Votes"] / max_votes) * 0.3

    # Sort by the new weighted score
    df_filtered = df_filtered.sort_values(by="Weighted_Score", ascending=False)

    return df_filtered["Movie Name"].head(top_n).tolist()


In [ ]:
import numpy as np

def recommend_movies(movie_title, df, cosine_sim, top_n=10):
    # Convert input movie title to lowercase and strip spaces
    movie_title = movie_title.strip().lower()

    # Create a mapping of lowercase movie names to their indices
    indices = pd.Series(df.index, index=df["Movie Name"].str.lower()).drop_duplicates()

    if movie_title not in indices:
        return "Movie not found in dataset."

    idx = indices[movie_title]

    # Get similarity scores for all movies
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort movies based on similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the top N most similar movies (excluding the input movie)
    sim_scores = sim_scores[1:top_n+1]

    # Get movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Filter based on high ratings and votes
    # Instead of using loc with movie_indices, which might be from the original df index,
    # filter sim_scores to only include indices present in the current df index

    valid_movie_indices = [i for i in movie_indices if i in df.index]
    df_filtered = df.loc[valid_movie_indices].copy() # Use valid indices for filtering

    # Normalize ratings and votes
    df_filtered["Rating"] = df_filtered["Rating"].astype(float)
    df_filtered["Votes"] = df_filtered["Votes"].astype(int)

    max_rating = df_filtered["Rating"].max()
    max_votes = df_filtered["Votes"].max()

    df_filtered["Weighted_Score"] = (df_filtered["Rating"] / max_rating) * 0.7 + (df_filtered["Votes"] / max_votes) * 0.3

    # Sort by the new weighted score
    df_filtered = df_filtered.sort_values(by="Weighted_Score", ascending=False)

    return df_filtered["Movie Name"].head(top_n).tolist()


In [ ]:
recommend_movies("inception", df, cosine_sim, top_n=5)


['star wars: episode v - the empire strikes back',
 'eternal sunshine of the spotless mind',
 'old dads',
 'good burger']